# Chapter 12: Smart Pointers

Learn about smart pointers that provide additional functionality beyond regular references.

## What Are Smart Pointers?

Smart pointers are data structures that:
- Act like pointers
- Have additional metadata and capabilities
- Own the data they point to (unlike references)
- Implement `Deref` and `Drop` traits

## Box<T>: Heap Allocation

`Box<T>` stores data on the heap instead of the stack.

In [ ]:
fn main() {
    let b = Box::new(5);
    println!("b = {}", b);
    
    // Box is dropped when b goes out of scope
}

## Use Cases for Box<T>

1. When you have a type whose size can't be known at compile time
2. When you want to transfer ownership of large data without copying
3. When you want to own a value that implements a specific trait

## Recursive Types with Box

In [ ]:
#[derive(Debug)]
enum List {
    Cons(i32, Box<List>),
    Nil,
}

use List::{Cons, Nil};

fn main() {
    let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
    println!("{:?}", list);
}

## Deref Trait

The `Deref` trait allows you to customize the `*` dereference operator.

In [ ]:
use std::ops::Deref;

struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        MyBox(x)
    }
}

impl<T> Deref for MyBox<T> {
    type Target = T;
    
    fn deref(&self) -> &Self::Target {
        &self.0
    }
}

fn main() {
    let x = 5;
    let y = MyBox::new(x);
    
    assert_eq!(5, x);
    assert_eq!(5, *y);
}

## Deref Coercion

Rust automatically converts references to types that implement `Deref`.

In [ ]:
fn hello(name: &str) {
    println!("Hello, {}!", name);
}

fn main() {
    let m = MyBox::new(String::from("Rust"));
    hello(&m);  // MyBox<String> -> &String -> &str
}

## Drop Trait

The `Drop` trait specifies code to run when a value goes out of scope.

In [ ]:
struct CustomSmartPointer {
    data: String,
}

impl Drop for CustomSmartPointer {
    fn drop(&mut self) {
        println!("Dropping CustomSmartPointer with data `{}`!", self.data);
    }
}

fn main() {
    let c = CustomSmartPointer {
        data: String::from("my stuff"),
    };
    let d = CustomSmartPointer {
        data: String::from("other stuff"),
    };
    println!("CustomSmartPointers created.");
    
    // Drop manually with std::mem::drop
    drop(c);
    println!("CustomSmartPointer dropped before end of main.");
}

## Rc<T>: Reference Counting

`Rc<T>` enables multiple ownership by keeping track of references.

In [ ]:
use std::rc::Rc;

enum List {
    Cons(i32, Rc<List>),
    Nil,
}

use List::{Cons, Nil};

fn main() {
    let a = Rc::new(Cons(5, Rc::new(Cons(10, Rc::new(Nil)))));
    println!("count after creating a = {}", Rc::strong_count(&a));
    
    let b = Cons(3, Rc::clone(&a));
    println!("count after creating b = {}", Rc::strong_count(&a));
    
    {
        let c = Cons(4, Rc::clone(&a));
        println!("count after creating c = {}", Rc::strong_count(&a));
    }
    
    println!("count after c goes out of scope = {}", Rc::strong_count(&a));
}

## RefCell<T>: Interior Mutability

`RefCell<T>` allows mutating data even when there are immutable references to it.

In [ ]:
use std::cell::RefCell;

fn main() {
    let data = RefCell::new(5);
    
    *data.borrow_mut() += 1;
    
    println!("data: {:?}", data.borrow());
}

## Combining Rc<T> and RefCell<T>

In [ ]:
use std::cell::RefCell;
use std::rc::Rc;

#[derive(Debug)]
enum List {
    Cons(Rc<RefCell<i32>>, Rc<List>),
    Nil,
}

use List::{Cons, Nil};

fn main() {
    let value = Rc::new(RefCell::new(5));
    
    let a = Rc::new(Cons(Rc::clone(&value), Rc::new(Nil)));
    let b = Cons(Rc::new(RefCell::new(3)), Rc::clone(&a));
    let c = Cons(Rc::new(RefCell::new(4)), Rc::clone(&a));
    
    *value.borrow_mut() += 10;
    
    println!("a = {:?}", a);
    println!("b = {:?}", b);
    println!("c = {:?}", c);
}

## Reference Cycles and Memory Leaks

It's possible to create memory leaks with `Rc<T>` and `RefCell<T>`.

In [ ]:
use std::cell::RefCell;
use std::rc::Rc;

#[derive(Debug)]
enum List {
    Cons(i32, RefCell<Rc<List>>),
    Nil,
}

impl List {
    fn tail(&self) -> Option<&RefCell<Rc<List>>> {
        match self {
            Cons(_, item) => Some(item),
            Nil => None,
        }
    }
}

// This creates a reference cycle!
// a -> b -> a -> b -> ...

## Weak<T>: Preventing Reference Cycles

`Weak<T>` creates non-owning references that don't prevent deallocation.

In [ ]:
use std::cell::RefCell;
use std::rc::{Rc, Weak};

#[derive(Debug)]
struct Node {
    value: i32,
    parent: RefCell<Weak<Node>>,
    children: RefCell<Vec<Rc<Node>>>,
}

fn main() {
    let leaf = Rc::new(Node {
        value: 3,
        parent: RefCell::new(Weak::new()),
        children: RefCell::new(vec![]),
    });
    
    let branch = Rc::new(Node {
        value: 5,
        parent: RefCell::new(Weak::new()),
        children: RefCell::new(vec![Rc::clone(&leaf)]),
    });
    
    *leaf.parent.borrow_mut() = Rc::downgrade(&branch);
    
    println!("leaf parent = {:?}", leaf.parent.borrow().upgrade());
}

## Exercises

1. Create a binary tree using Box<T>
2. Implement a doubly-linked list with Rc<T> and RefCell<T>
3. Use Weak<T> to prevent reference cycles in a graph
4. Implement Drop for a custom type

In [ ]:
// Exercise: Binary tree
struct TreeNode {
    value: i32,
    left: Option<Box<TreeNode>>,
    right: Option<Box<TreeNode>>,
}

impl TreeNode {
    fn new(value: i32) -> Self {
        TreeNode {
            value,
            left: None,
            right: None,
        }
    }
    
    fn insert_left(&mut self, value: i32) {
        // Your code here
    }
}

## Key Takeaways

- `Box<T>` allocates data on the heap
- `Rc<T>` enables multiple ownership through reference counting
- `RefCell<T>` provides interior mutability with runtime borrow checking
- `Weak<T>` prevents reference cycles
- Smart pointers implement `Deref` and `Drop` traits
- Deref coercion makes smart pointers convenient to use
- Be careful of reference cycles when combining `Rc<T>` and `RefCell<T>`